In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt =  ChatPromptTemplate.from_messages(
    
    [
        ("system" , "You are world class technical documentation writer"),
        ("user", "{input}"),
        
        
    ]
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
chain = prompt | model | output_parser

In [ ]:
print(chain.invoke({
    
    "input" : "What is LAngGraph?"
}))

# Retrieval Technique 
*Modeli beslemek için veriler olduğu zaman kullanabiliriz.(PDF, Database, WebSite)
* Bilgiler Vector veritabanında tutarız
* Örnek olarak Web sitesi için verileri indexlememiz lazım. Bunun için 
WebBaseLoader (BeautifulSoup Library) kullanırız

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("kgtu.edu.tr")

docs = loader.load()

In [ ]:
docs #alınan site

In [ ]:
#Embedding işlemi 
from langchain_openai import OpenAıEmbeddings
emneddings = OpenAıEmbeddings()
#faiss kullanılacak
from langchain_community.vectorstores import FAISS

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs) #metinler parçalandı

vector = FAISS.from_documents(documents, embeddings) # veritabanını indexliyoruz.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    """
    Answer the followiwng question based only on the provided context.
    <context>
    {context}
    </context>
    Question : {input}
    """
    
    )


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(model, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [ ]:
response = retrieval_chain.invoke(
    {
        "input" : "What is LangGraph"
    }
)
print(response)

In [ ]:
from langchain_core.documents import Document 
my_doc = Document(
    page_content = "LangGraph is a module built on top of Langchain :)"
)

In [ ]:
document_chain.invoke(
    {
        "input" : " What is LangGraph?",
        "context" : [my_doc],
    }
)